In [ ]:
%matplotlib inline
from pylab import *
import dataset
import fullyconn
from IPython import display

# Loading data

In [ ]:
data = dataset.load_dataset()
train_data = data[:-len(data)/2]
train_data = np.reshape(train_data, (len(train_data), 3 * 40 * 40))
test_data = data[len(data)/2:]
test_data = np.reshape(test_data, (len(test_data), 3 * 40 * 40))

 # Model plan (fcc autoencoder 1600-30-1600 with MSE loss)

In [ ]:
def build_nn():
    print("Building NN")
    nn = fullyconn.MLP(40 * 40 * 3)
    nn.add_layer("bottleneck", 90)
    nn.add_layer("reconstruction", 40 * 40 * 3)
    return nn

# Evaluation function (test set)

In [ ]:
def precision(dataset, encode_decode):
    # L1 distance between original example and encoded-decoded example
    precision = np.absolute(dataset - encode_decode(dataset)).mean()
    return precision

# Reconstruction visualisation

In [ ]:
import Image
def show_reconstruction(example, encode_decode):
    reconstructed = encode_decode(np.array([example]))
    
    example = np.array(example.reshape(40, 40, 3), dtype="float")
    reconstructed = np.array(reconstructed.reshape(40, 40, 3), dtype="float")
    
    figure()
    subplot(121)
    imshow(example, interpolation="nearest")
    subplot(122)
    imshow(reconstructed, interpolation="nearest")

# Model training

In [ ]:
def train_network(nn, learning_rate, regularization, batch_size, steps):
    test_epoch = []
    train_epoch = []
    train = nn.build_train(learning_rate, regularization)
    eval  = nn.build_eval()
    for step_id in range(steps):
        if step_id % 10 == 0:
            display.clear_output(wait=True)
            test_precision = precision(test_data, eval)
            train_precision = precision(train_data, eval)
            
            train_epoch.append(train_precision * 100)
            test_epoch.append(test_precision * 100)
            
            x = linspace(0, step_id, len(train_epoch))
            
            plot(x, test_epoch, 'r')
            plot(x, train_epoch, 'b')
            for i in range(5):
                show_reconstruction(test_data[i], eval)
            show()
            

        for batch_id in range(0, len(train_data) / batch_size):
            ts = train_data[batch_id * batch_size: (batch_id + 1) * batch_size] 

            train(ts, ts)

In [ ]:
nn = build_nn()

In [ ]:
train_network(nn, 0.005, 0.0, 10, 2000)

In [ ]:
train_network(nn, 0.005, 0.01, 10, 2000)